# Transfer Learning


### Celem jest zapoznanie się z dostępnymi modelami (architekturami).

### Modele
Jak widać Keras ma trochę dostępnych modeli. Jeśli chcesz poczytać więcej o każdym z nich, poniżej są linki:

- [ResNet50: Deep Residual Learning for Image Recognition](https://bit.ly/3b61MLt)
- [VGG16: Very Deep Convolutional Networks for Large-Scale Image Recognition](https://bit.ly/3xYwjVs)
- [VGG19: Very Deep Convolutional Networks for Large-Scale Image Recognition](https://bit.ly/3xYwjVs)
- [InceptionResNetV2: Inception-v4, Inception-ResNet and the Impact of Residual Connections on Learning](https://bit.ly/3heUn07)
- [InceptionV3: Rethinking the Inception Architecture for Computer Vision](https://bit.ly/3o0LJnw)
- [MobileNet: Efficient Convolutional Neural Networks for Mobile Vision Applications](https://bit.ly/2RA1PrU)
- [Xception: Deep Learning with Depthwise Separable Convolutions](https://bit.ly/3vJpvJ8)

Weźmiemy na początek jeden z prostszych, ale dość popularny - **VGG16**.

*Swoją drogą*, wczytując model, mamy flagę `include_top=False`, która oznacza, że obcinamy warstwę klasyfikatora (czyli `fully-connected layer`). Na początek wczytamy całość (razem z `fully-connected layers`).

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPool2D

from tensorflow.keras.optimizers import Adam, SGD

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.densenet import DenseNet121, DenseNet169, DenseNet201

from tensorflow.keras.preprocessing import image

import numpy as np
np.random.seed(0)

## VGG16

In [4]:
model = VGG16(weights='imagenet', include_top=True)
model.summary()

2022-11-11 15:43:35.968464: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


553467904/553467096 [==============================] - 23s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
___________________________________________________________

Jaki widać VGG16 ma ponad 138M parametrów (bez `fully-connected` ma 14M i trochę, czyli 10 razy mniej). Domyślnie wszystkie warstwy są "odmrożone" (zwróć uwagę na ostatnią linię poprzedniej komórki: `Non-trainable params: 0`).

Do warstw można dostać się w taki sposób: `model.layers`. Każda warstwa ma atrybut `trainable`, który zwraca `True` lub `False`. Sprawdźmy to.

In [5]:
for it, layer in enumerate(model.layers):
    print(it, layer.trainable)

0 True
1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 True
9 True
10 True
11 True
12 True
13 True
14 True
15 True
16 True
17 True
18 True
19 True
20 True
21 True
22 True


Pierwsza warstwa już jest ustawiona na `False`, bo jest to `InputLayer`. Załóżmy, że chcemy trenować (ang. *fine tuning*) tylko ostatni (5. blok), który składa się z czterech warstw (niskopoziomowych):

```
block5_conv1 (Conv2D)     (None, None, None, 512)   2359808   
block5_conv2 (Conv2D)     (None, None, None, 512)   2359808   
block5_conv3 (Conv2D)     (None, None, None, 512)   2359808   
block5_pool (MaxPool2D)   (None, None, None, 512)   0 
```


W takiej sytuacji można dla reszty warstw ustawić parametr `trainable` na `False`.

In [6]:
for it, layer in enumerate(model.layers[:-8]):
    layer.trainable = False
for it, layer in enumerate(model.layers[-4:]):
    layer.trainable = False

Sprawdzamy całość i potem jeszcze `summary` (zwracamy uwagę na ostatnią linię - `Non-trainable params`).

In [7]:
for it, layer in enumerate(model.layers):
    print(it, layer.name, layer.trainable)

0 input_1 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 True
16 block5_conv2 True
17 block5_conv3 True
18 block5_pool True
19 flatten False
20 fc1 False
21 fc2 False
22 predictions False


In [8]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [9]:
model = VGG16(weights='imagenet', include_top=False)
model.summary()

58892288/58889256 [==============================] - 4s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
______________________________________________________________

## Przypadki zadań 
W dużym uproszczeniu wszystkie zadania w Deep Learning może klasyfikować wg dwóch wymiarów:
![](../images/four_cases.png)

A teraz przejdziemy po kolei po każdym przypadku.

### (1) Danych jest sporo i zadanie bardzo się różni.

W takiej sytuacji tak naprawdę uczymy naszą sieć od zera (czyli `ImageNet` trafia do tego przypadku, kiedy mamy miliony zdjęć i zupełnie nowe zadanie).

### (2) Danych jest dużo, ale zadanie jest podobne.

Skoro mamy dużo danych, to możemy uczyć się nawet od zera (przynajmniej nie powinien nam grozić `overfitting`). Z drugiej strony, skoro zadanie jest podobne, to warto wykorzystać już wyliczone wagi. 

Dlatego strategia jest taka - odmrażamy wszystkie warstwy (`trainable=True`), to jest opcja domyślna. Jedynie jeszcze można przerobić `fully-connected layer` - chyba, że to też nam odpowiada (np. `ImageNet` ma 1000 klas na wyjściu).

In [10]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

for layer in base_model.layers[1:]: #input layer ma być na False
    layer.trainable = True


model = Sequential([
    base_model,
    
    Flatten(), #<= bridge between conv layers and full connected layers
    
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
    
])

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

Sprawdzamy, jak to jest.

In [11]:
for layer in model.layers:
    print(layer.name, layer.trainable)
    
model.summary()

vgg16 True
flatten True
dense True
dropout True
dense_1 True
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 1024)              525312    
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                10250     
Total params: 15,250,250
Trainable params: 15,250,250
Non-trainable params: 0
_________________________________________________________________


Zwróć uwagę, że VGG16 teraz jest pokazany jako warstwa - jest to dość wygodne, w `summary` widać tylko to, co jest ważne (nasze).

### (3) Danych jest mało, ale i zadanie jest podobne.

W sytuacji gdy danych jest mało lub wcale ich nie ma, to musimy użyć takiego modelu, jaki jest (właśnie to już zrobiliśmy w punkcie `2`) albo zamrozić wszystkie warstwy konwolucyjne i wyrzucić lub odmrozić `fully-connected layers`.

In [12]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

for layer in base_model.layers[1:]: #input layer ma być na False
    layer.trainable = False #!!!! zwróć uwagę, zamrażamy wszystkie warstwy


model = Sequential([
    base_model,
    
    Flatten(), #<= bridge between conv layers and full connected layers
    
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
    
])

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

Sprawdzamy, jak to jest:

In [13]:
for layer in model.layers:
    print(layer.name, layer.trainable)
    
model.summary()

vgg16 True
flatten_1 True
dense_2 True
dropout_1 True
dense_3 True
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                10250     
Total params: 15,250,250
Trainable params: 535,562
Non-trainable params: 14,714,688
_________________________________________________________________


W drugim przypadku było `Trainable params: 15,250,250`, a w tym przypadku jest: `Trainable params: 535,562`. Trenujemy tylko `fully-connected layer` i nawet mając mało danych jest szansa, że nie będziemy "overfitować".

### (4) Danych jest mało i zadanie (bardzo) się różni.
To jest najgorszy przypadek i niestety dość częsty :) Trzeba sobie jakoś z tym radzić. W tej sytuacji odcięcie tylko `full-connected layer` to raczej za mało - trzeba odmrozić coś więcej (kilka lub więcej warstw konwolucyjnych). Natomiast możemy tutaj zastosować dość sprytne zagranie, które zwykle sprawdza się w praktyce.

Na początek zamrażamy wszystkie warstwy konwolucyjne i trenujemy tylko `fully-connected layer`, a następnie odmrażamy wszystko, tylko trenujemy z mniejszym krokiem.

### Faza pierwsza
Zamrażamy wszystkie warstwy konwolucyjne i trenujemy tylko `fully-connected layer`.

In [14]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

for layer in base_model.layers[1:]: #input layer ma być na False
    layer.trainable = False #!!!! zwróć uwagę, zamrażamy wszystkie warstwy


model = Sequential([
    base_model,
    
    Flatten(), #<= bridge between conv layers and full connected layers
    
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
    
])

optimizer = Adam(0.001, decay=0.0003) #wartości są przykładowe, zwróć uwagę tylko na rząd
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

for layer in model.layers:
    print(layer.name, layer.trainable)
    
model.summary()

vgg16 True
flatten_2 True
dense_4 True
dropout_2 True
dense_5 True
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                10250     
Total params: 15,250,250
Trainable params: 535,562
Non-trainable params: 14,714,688
_________________________________________________________________


### Faza druga

Odmrażamy wszystkie lub część warstw z VGG. Swoją drogą, żeby się tam dostać trzeba odwołać się do pierwszej warstwy `model.layers[0]`.

In [15]:
for layer in model.layers[0].layers[10:]: 
    layer.trainable = True

optimizer = Adam(0.0001, decay=0.00000001) #wartości są przykładowe, zwróć uwagę tylko na rząd
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
for layer in model.layers[0].layers:
    print(layer.name, layer.trainable)
    
model.summary()

input_5 True
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool True
block4_conv1 True
block4_conv2 True
block4_conv3 True
block4_pool True
block5_conv1 True
block5_conv2 True
block5_conv3 True
block5_pool True
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
___________________________________________

## Przydatne linki:
1. [CS231n: Transfer Learning](https://bit.ly/2RDIBBy)
2. [Learning Keras by Implementing the VGG Network From Scratch](https://bit.ly/2Q4uZPG)
3. [Experiments on different loss configurations for style transfer](https://bit.ly/3vQP7DU)
4. [Transfer Learning using Keras](https://bit.ly/3nYe0ed)
5. [A Comprehensive Hands-on Guide to Transfer Learning with Real-World Applications in Deep Learning)](https://bit.ly/3b9zAYb)